# Fluid‑HD Decoding & Results

In this notebook we:
1. Load saved `hop_log` and `M_memory`.
2. Decode top‑k hop partners for each agent.
3. Compute Recall@1 and Recall@5.
4. Plot metrics vs. HD dimension and affinity gap.
5. Visualize true vs. decoded hop matrices and SNR curves.

## 2.1. Load Results

In [ ]:
# Analysis parameters
D           = 4096
tau_intra   = 0.85
tau_inter   = 0.35
top_k       = [1,5]

log_path    = '../results/hop_log.pkl'
mem_path    = '../results/M_memory.npz'

## 2.2. Decode Partners

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from binding    import HDBinder
from memory     import MemoryStore
from decoder    import decode_partners
from utils      import compute_recall, compute_snr

# load hop log & memory
with open(log_path,'rb') as f: hop_log = pickle.load(f)
M = np.load(mem_path)['M']

# reconstruct E_id (same seed & D)
binder = HDBinder(N=100, D=D, seed=seed)
E_id   = binder.E_id

## 2.3. Performance Analysis

In [ ]:
from collections import Counter
gt_counts = Counter((i,j) for (_,i,j) in hop_log)

## 2.4. Visualizations

In [ ]:
decoded = decode_partners(M, E_id, top_k=5) # dict i→list of js
recalls = {k: compute_recall(decoded, gt_counts, k) for k in top_k}
print('Recall@1 =', recalls[1], 'Recall@5 =', recalls[5])

## 2.5. Agent Trajectory Animation (First 200 Steps)

In [ ]:
# load metrics.csv if exists
df = pd.read_csv('../results/metrics.csv')
df['tau_gap'] = df['tau_intra'] - df['tau_inter']
sns.lineplot(data=df.query('tau_gap==0.5'), x='D', y='recall1', marker='o')
plt.xlabel('HD Dimension D')
plt.ylabel('Recall@1')
plt.title('Recall@1 vs D (gap=0.5)')
plt.show()

## 2.6. Heatmap: True vs Decoded Partners

In [ ]:
# Create matrices for visualization
true_matrix = np.zeros((N, N))
decoded_matrix = np.zeros((N, N))

for i in range(N):
    for j in true_partners[i]:
        true_matrix[i, j] = 1
    for j in decoded_partners_5[i]:
        decoded_matrix[i, j] = 1

# Create the heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# True partners heatmap
im1 = ax1.imshow(true_matrix, cmap='Blues', aspect='equal')
ax1.set_title('True Interaction Partners')
ax1.set_xlabel('Agent ID')
ax1.set_ylabel('Agent ID')
plt.colorbar(im1, ax=ax1)

# Decoded partners heatmap
im2 = ax2.imshow(decoded_matrix, cmap='Reds', aspect='equal')
ax2.set_title('Decoded Top-5 Partners')
ax2.set_xlabel('Agent ID')
ax2.set_ylabel('Agent ID')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

# Calculate overlap
overlap = np.sum(true_matrix * decoded_matrix)
total_true = np.sum(true_matrix)
total_decoded = np.sum(decoded_matrix)

print(f"Overlap between true and decoded: {overlap}")
print(f"Total true interactions: {total_true}")
print(f"Total decoded interactions: {total_decoded}")
print(f"Precision: {overlap/total_decoded:.3f}")
print(f"Recall: {overlap/total_true:.3f}")

In [ ]:
sns.lineplot(data=df.query('D==4096'), x='tau_gap', y='recall1', marker='o')
plt.xlabel('Affinity Gap')
plt.ylabel('Recall@1')
plt.title('Recall@1 vs Affinity Gap (D=4096)')
plt.show()

In [ ]:
# build matrices
N = 100
true_mat    = np.zeros((N,N))
decoded_mat = np.zeros((N,N))
for (i,j),c in gt_counts.items(): true_mat[i,j]=c
for i,js in decoded.items(): decoded_mat[i,js]=1

fig, axes = plt.subplots(1,2, figsize=(10,4))
sns.heatmap(true_mat, ax=axes[0], cmap='magma'); axes[0].set_title('True')
sns.heatmap(decoded_mat, ax=axes[1], cmap='magma'); axes[1].set_title('Decoded')
plt.show()

In [ ]:
snrs = [compute_snr(n, D, len(hop_log)) for (_,n) in gt_counts.most_common(20)]
plt.bar(range(20), snrs)
plt.xlabel('Top pairs')
plt.ylabel('SNR')
plt.title('SNR of Top‑20 Hop Pairs')
plt.show()